In [ ]:
from collections import namedtuple
import glob
import re
import random
import random
import cv2
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.notebook import trange, tqdm
from deep_shadow import *
from utils import *
import itertools
from typing import NamedTuple
import predict


%load_ext autoreload
%autoreload 2

In [ ]:
# deep_shadow = DeepShadow(512, 512)
# deep_shadow.restore('training_checkpoints/evaluation_new/uniform_cities')

In [ ]:
height_folder = Path('data/heights_new')
output_folder = Path('data/shadows_new')

In [ ]:
predict.predict_cities(height_folder, output_folder)
# predict.predict_at_city_zoom()
"""
0   1   2   3
4   5   6   7
8   9   10  11
12  13  14  15
"""

In [ ]:
import numpy as np

x = 0
y = 0
indices = [
    (xx, yy)
    for xx in range(256+256*x, 256+256*(x+1))
    for yy in range(256+256*y, 256+256*(y+1))
]
indices = np.array(indices).reshape(256,256,-1)
indices

In [6]:
indices.shape

(256, 256, 2)

In [22]:
import numpy as np
x = 0
y = 0
a, b = np.meshgrid(
    range(256+256*x, 256+256*(x+1)),
    range(256+256*y, 256+256*(y+1)),
)

In [29]:
a, b = np.meshgrid(
    np.arange(256+256*x, 256+256*(x+1)),
    np.arange(256+256*y, 256+256*(y+1)).reshape((1,-1,1))
)

In [37]:
np.concatenate((a,b), axis=0).reshape((1,-1,2))

array([[[256, 257],
        [258, 259],
        [260, 261],
        ...,
        [511, 511],
        [511, 511],
        [511, 511]]])

In [31]:
b

array([[256, 256, 256, ..., 256, 256, 256],
       [257, 257, 257, ..., 257, 257, 257],
       [258, 258, 258, ..., 258, 258, 258],
       ...,
       [509, 509, 509, ..., 509, 509, 509],
       [510, 510, 510, ..., 510, 510, 510],
       [511, 511, 511, ..., 511, 511, 511]])

In [54]:
import numpy as np
annoying = [
    (xx, yy)
    for xx in range(256+256*x, 256+256*(x+1))
    for yy in range(256+256*y, 256+256*(y+1))
]
annoying = np.array(indices).reshape((256,256,-1))
from sklearn.utils.extmath import cartesian
easy = cartesian((
    range(256+256*x, 256+256*(x+1)),
    range(256+256*y, 256+256*(y+1)),
)).reshape(256,256,-1)

(easy == annoying).all()

True